In [ ]:
!pip install transformers
!pip install accelerate
!pip install OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
from openai import OpenAI
import pickle, sys, copy, re, math
import pandas as pd
import concurrent.futures
import re
import time
import numpy as np

In [ ]:
#mounting google drive to avoid uploading files each runtime
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
#Read the csv dataset as a pandas dataframe
path='/content/drive/MyDrive/Data/claudette_train_multi_label.csv'
df_train = pd.read_csv(path)
#Print to check if this is the correct dataset
df_train.head()

Mounted at /content/drive


,Index,Document,Labels,Sentence
0,0,Mozilla,[],websites & communications terms of use
1,1,Mozilla,[],please read the terms of this entire document ...
2,2,Mozilla,['USE'],by accessing or signing up to receive communic...
3,3,Mozilla,[],our websites include multiple domains such as ...
4,4,Mozilla,[],you may also recognize our websites by nicknam...


In [ ]:
#Read the csv dataset as a pandas dataframe
feature_name = 'legalbert' # 'sbert' or 'legalbert'
path='/content/drive/MyDrive/Data/'+feature_name+'.pkl'
features = np.load(path, allow_pickle=True)

In [ ]:
print(features[0:5])

[[-0.11907776 -0.51734316 -0.20452373 ...  0.09718019  0.00218152
   0.02430376]
 [-0.1980409  -0.28219828 -0.09045833 ... -0.30171952  0.22982122
   0.23000887]
 [-0.21410233 -0.77765685 -0.2686013  ... -0.15008977  0.18910606
   0.06108784]
 [ 0.11285687 -0.04988806  0.5990627  ...  0.70457274 -0.02006482
  -0.416788  ]
 [-0.01490193 -0.02150737  0.4988497  ...  0.75571233 -0.60508513
   0.0941601 ]]


In [ ]:
#trim features to get ONLY training data and append embeddings
selected_features_train = features[df_train["Index"].tolist()]
selected_features_train=pd.DataFrame(selected_features_train)

dummy=selected_features_train.values.tolist()
df_train["LegalBert"]=dummy

In [ ]:
df_train.tail()

,Index,Document,Labels,Sentence,LegalBert
8349,20297,Foursquare,[],solely in respect of users located in the eea ...,"[-0.13980035483837128, -0.14969150722026825, -..."
8350,20298,Foursquare,[],where foursquare has provided you with a trans...,"[-0.018744483590126038, -0.4091951847076416, -..."
8351,20299,Foursquare,[],if there is any contradiction between what the...,"[-0.25013822317123413, -0.5396697521209717, 0...."
8352,20300,Foursquare,[],you may contact foursquare at the following ad...,"[-0.18877653777599335, -0.2694542706012726, -0..."
8353,20301,Foursquare,[],"effective date : january 29 , 2013","[-0.08851226419210434, -0.18286660313606262, 0..."


In [ ]:


def get_closest_negative(embedding1, df, label):
    query = embedding1
    df_train_neg = df[~df["Labels"].str.contains(label)]

    # Stack the embeddings into a 2D array
    embeddings_stack = np.stack(df_train_neg['LegalBert'].to_numpy())

    # Calculate cosine similarity in batches
    dot_products = np.dot(query, embeddings_stack.T)
    query_norm = np.linalg.norm(query)
    features_norms = np.linalg.norm(embeddings_stack, axis=1)
    similarities = dot_products / (query_norm * features_norms)

    # Use argmax to get the index of the top similarity
    top_index = np.argmax(similarities)

    return df.iloc[top_index]

#test
test_embed = features[2]
print('Closest negative:',get_closest_negative(test_embed,df_train,'USE'))

Closest negative: Index                                                    14135
Document                                                Grindr
Labels                                                      []
Sentence     anytime that we decide to charge you for a por...
LegalBert    [-0.1489204615354538, -0.34428897500038147, 0....
Name: 6245, dtype: object


In [ ]:
#Enter the label as its capitalized acronym
#Returns a sampled df with half positives and half closest negatives
#No random seed because we want the sentences to change between calls for the evaluation
#We return a df for ease of future use

def sample_n_clauses(df, n, label):
    df_pos = df[df["Labels"].apply(lambda x: label in x)]

    sampled_neg = pd.DataFrame()

    sampled_pos = df_pos.sample(n//2, replace=False)

    for index in sampled_pos.index:
        pos_sent = sampled_pos['LegalBert'][index]
        neg_sent = get_closest_negative(pos_sent, df, label)
        sampled_neg = pd.concat([sampled_neg, neg_sent.to_frame().T], ignore_index=True)

    # Reset the index of sampled_neg before concatenating
    sampled_neg = sampled_neg.reset_index(drop=True)

    result_df = pd.concat([sampled_pos, sampled_neg], axis=0, ignore_index=True)

    return result_df
sampled_df= sample_n_clauses(df_train, 20,'J')
# print(sampled_df)


In [ ]:
#Query to mistral API:
openai = OpenAI(
    api_key="XqPUITUxARnzuvDPvLWMQW86dKIAS1Ih",
    base_url="https://api.deepinfra.com/v1/openai",
)

def query_mistral(prompt, temperature=0.0, max_tokens = 50, role='user'):
    chat_completion = openai.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.1",
        messages=[{"role": role, "content": prompt},
                  {"role": "system",
                 "content": "[INST] Limit your response to 50 words.[INST]"}
            ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return chat_completion.choices[0].message.content

print(query_mistral("Hello"))

 Hello! How can I assist you today?


In [ ]:
#Compare the answer of the classification LLM to the true label to find false postives and false negatives
def is_fpnp(answer,true_label):
  if "yes" in answer.lower() and true_label=="positive":
    return "tp"
  elif "no" in answer.lower() and true_label=="positive":
    return "fn"
  elif "yes" in answer.lower() and true_label=="negative":
    return "fp"
  elif "no" in answer.lower() and true_label == "negative":
    return "tn"
  return "tn"
#Tests
print("tp expected : ",is_fpnp("YES","positive"))
print("fp expected : ",is_fpnp("YES","negative"))
print("fn expected : ",is_fpnp("NO","positive"))
print("tn expected : ",is_fpnp("NO","negative"))

tp expected :  tp
fp expected :  fp
fn expected :  fn
tn expected :  tn


In [ ]:
def get_GT(row,label_class):

  if label_class in row[2]:
    return 'positive'
  return 'negative'
#test
print(get_GT(sampled_df.iloc[19],'J'))

negative


In [ ]:
def get_f1(tp, fp, fn):
    f1_score = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 0
    return f1_score

#test
print("F1 Score (direct calculation):",get_f1(tp=10, fp=2, fn=2))


F1 Score (direct calculation): 0.8333333333333334


# Expand & Select





In [ ]:

#Should return a list of sublist [["fp","clause","prompt","answer"],....] only return fp and fn since we won't keep the others
# Evaluation of one prompt across a minibatch, saving fn and fp prompts


def evaluate(original_prompt,minibatch_df,label):
  results=[]
  for row in minibatch_df.iterrows():
    cache={}

    true_label = ""

    if label in row[1][2]:
      true_label="positive"
    else:
      true_label="negative"

    clause=row[1][3]
    global_prompt = f'Consider the following online terms of service clause: {clause} \n {original_prompt} \n Start your answer with "yes" or "no" and then justify your response in no more than 50 words.'

    answer = query_mistral(global_prompt)

    status=is_fpnp(answer,true_label) #returns the tp/fp/tn/fn string


    if status == "fp" or status == "fn":
      cache["status"]=status
      cache["clause"]=clause
      cache["prompt"]=original_prompt
      cache["answer"]=answer
      cache ['gt']=true_label

      #generate critique
      if status == "fp":
        cache['prediction']="Yes"
      if status=="fn":
        cache['prediction']="No"


      results.append(cache)

      #print('RESULT DICT:',cache)

  return results



In [ ]:
def get_critiq(prompt_dict):
  original_prompt=prompt_dict['prompt']
  prediction=prompt_dict['prediction']
  answer=prompt_dict['answer']
  true_label=prompt_dict['gt']


  critique_string= f"Online service contracts can contain clauses that may be deemed potentially unfair under European Law. Our goal is to correctly classify jurisdiction clauses <OLD instruction>"\
                      +f"{original_prompt}<\\OLD instruction> Given instruction produced this prediction : {prediction}"\
                      +f"with this reasoning{answer}"\
                      +f", True label is: +{true_label}"\
                      +"[INST] Analyse and provide 3 constructive feedback points on the prompt. Your answer should not be longer than 50 words.[\\INST]"

  critiq=query_mistral(critique_string,temperature=1)

  return critiq


In [ ]:
# #Expand function from the paper
# #We first evaluate the prompt over a minibatch, saving fp and fn
# def expand(prompt,n,label,training_df):
#   print(f'Expanding the beam ..... for prompt : {prompt} \n')
#   minibatch_df = sample_n_clauses(training_df,n,label)
#   results = evaluate(prompt,minibatch_df,label)
#   candidates = []
#   filtered = []
#   target=""

#   fp=0
#   fn=0
#   for error in results :
#     if error["status"]=="fp":
#       fp+=1
#     elif error ["status"]=="fn":
#       fn+=1
#   print(f"Number of fp : {fp} , Number of fn : {fn}")


#   #for each fp or fn we craft a meta prompt to obtain a new (hopefully better prompt)
#   for error in results:
#     critiq=get_critiq(error)
#     g_meta=""
#     if error["status"]=="fp":
#       g_meta = 'The model incorrectly answers "yes", but should answer "no", as the clause is considered fair under this standard.'
#       target = "No"
#     else:
#       g_meta='The model incorrectly answers "no", but should answer "yes", as the clause is considered fair under this standard.'
#       target="Yes"
#     meta_prompt = f'Consider the following online terms of service clause: "{error["clause"]}"\n'\
#                   +f'We run this Unfairness detection instruction: <instr>{prompt}</instr>\n'\
#                   +g_meta+'\n'\
#                   +f'This instruction is critiqued here: "{critiq}"\n'\
#                   +f'[INST]Craft a short and revised instruction framed by <instr> ... </instr> tags that remedies the model\'s misunderstanding. Your instruction must not refers to the clause. End instruction on a question that leads to the correct "{target}" answer[\\INST]:\n'
#     new_prompt=query_mistral(meta_prompt,1.1)
#     candidates.append(new_prompt)
#     #print('META PROMPT:'+meta_prompt)


#     # print("number of candidates : ", len(candidates))
#     #Filtering of the results (cf Matthias Notebook)

#     for c in candidates:
#         extraction = re.findall("<instr>(.*?)</instr>", c, re.DOTALL)
#         if len(extraction) == 1:
#             filtered.append(extraction[0].strip())



#   print(f'Candidates : ',candidates)
#   print(f'filtered : ', filtered)
#   return candidates


# #test

# print(expand("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",4,'J',df_train))


In [ ]:
def evaluate_data_parallel(data, prompt):
    return evaluate(prompt, data)

def generate_candidates(error, prompt):
    critiq = get_critiq(error)
    g_meta = ""
    target = ""

    if error["status"] == "fp":
        g_meta = 'The model incorrectly answers "yes", but should answer "no", as the clause is considered fair under this standard.'
        target = "No"
    else:
        g_meta = 'The model incorrectly answers "no", but should answer "yes", as the clause is considered unfair under this standard.'
        target = "Yes"

    meta_prompt = f'Consider the following online terms of service clause: "{error["clause"]}"\n' \
                  + f'We run this Unfairness detection instruction: <instr>{prompt}</instr>\n' \
                  + g_meta + '\n' \
                  + f'This instruction is critiqued here: "{critiq}"\n' \
                  + f'[INST]Craft a short and revised instruction  that remedies the model\'s misunderstanding. Your instruction must not refer to the clause. End instruction on a question that leads to the correct "{target}" answer[\\INST]:\n'

    new_prompt = query_mistral(meta_prompt, 1.1)
    return new_prompt

def expand_parallel(prompt, n, label, training_df):
    print(f'Expanding the beam ..... for prompt : {prompt} \n')
    minibatch_df = sample_n_clauses(training_df, n, label)
    results = evaluate(prompt, minibatch_df,label)
    candidates = []

    fp = 0
    fn = 0
    for error in results:
        if error["status"] == "fp":
            fp += 1
        elif error["status"] == "fn":
            fn += 1
    print(f"Number of fp : {fp} , Number of fn : {fn}")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []

        # Generate candidates in parallel
        for error in results:
            future = executor.submit(generate_candidates, error, prompt)
            futures.append(future)

        concurrent.futures.wait(futures)

        # Get the results
        for future in futures:
            candidates.append(future.result())

    filtered = [extraction[0].strip() for c in candidates for extraction in re.findall("<instr>(.*?)</instr>", c, re.DOTALL) if len(extraction) == 1]

    print(f'Candidates : ', candidates)
    print(f'filtered : ', filtered)
    return candidates

# #SPEED TEST
# start_time = time.time()
# result=expand("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",5)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The basic expand function took {select_time} s to complete its task")

# start_time = time.time()
# result=expand_parallel("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",5,'J',df_train)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The parallelized selection function took {select_time} s to complete its task")

In [ ]:
# def get_score(prompt,minibatch_df):

#   response_dict={'tp':0,'tn':0,'fp':0,'fn':0}

#   #evaluate prompts on entire minibatch
#   for data in minibatch_df.iterrows():
#     #print(data[1][3])
#     query_string=f'Consider the following online terms of service clause: {data[1][3]}'\
#                 +f'{prompt} \n Start your answer with "yes" or "no" and then justify your response in no more than 50 words.'

#     answer=query_mistral(query_string)
#     #print(answer)
#     res=is_fpnp(answer,get_GT(data[1],'J'))
#     response_dict[res]+=1

#   #compute F1 score
#   print(f"Scores for prompt : {prompt} : {response_dict}")
#   score=get_f1(response_dict['tp'],response_dict['fp'],response_dict['fn'])
#   #print(score)
#   return score

#test
#print(get_score("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",sampled_df))

In [ ]:
def evaluate_data(data, prompt, label):
    #print(data[1][3])
    query_string = f'Consider the following online terms of service clause: {data[1][3]}' \
                   + f'{prompt} \n Start your answer with "yes" or "no" and then justify your response in no more than 50 words.'

    answer = query_mistral(query_string)
    res = is_fpnp(answer, get_GT(data[1], label))

    return res

def get_score_parallel(prompt, label, minibatch_df):
    response_dict = {'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0}

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []

        for data in minibatch_df.iterrows():
            # Pass the 'label' argument here
            future = executor.submit(evaluate_data, data, prompt, label)
            futures.append(future)

        # Wait for all futures to complete
        concurrent.futures.wait(futures)

        # Get the results
        for future in futures:
            res = future.result()
            response_dict[res] += 1

    return get_f1(response_dict['tp'],response_dict['fp'],response_dict['fn'])

#SPEED TEST

# start_time = time.time()
# result=get_score("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",sampled_df)
# print(result)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The basic get_score function took {select_time} s to complete its task")

# start_time = time.time()
# result=get_score_parallel("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",'J',sampled_df)
# print(result)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The parallelized get_score function took {select_time} s to complete its task")

In [ ]:
#argmax function UCB algorithm

def get_argmax_UCB(list_NQ,t):

  for index in range(len(list_NQ)):
    n=list_NQ[index]['N']
    q=list_NQ[index]['Q']

    value=(q/n)+(2*(math.sqrt(math.log(t+1)/n)))
    list_NQ[index]['ucb_val']=value
    print(f'UCB value for prompt: {list_NQ[index]["prompt"]} is {list_NQ[index]["ucb_val"]}\n')


  sorted_listNQ=sorted(list_NQ, key=lambda x:x['ucb_val'],reverse=True)

  print(f'BEST UCB VAL: {sorted_listNQ[0]}')
  return sorted_listNQ[0]['prompt']

# #argmax function UCB_E algorithm
# def get_argmax_UCB_E(list_NQ):

#   for index in range(len(list_NQ)):
#     n=list_NQ[index]['N']
#     q=list_NQ[index]['Q']

#     value=(q/n)+(2*(math.sqrt(2/n)))
#     list_NQ[index]['ucb_val']=value

#   sorted_listNQ=sorted(list_NQ, key=lambda x:x['ucb_val'],reverse=True)

#   return sorted_listNQ[0]['prompt']

# #test

In [ ]:
#SELECT K top performing prompts

# def select_UCB_E(list_of_prompts,label,k,minibatchsize,num_iteration):
#   num_iteration=2*len(list_of_prompts)
#   num_prompts = len(list_of_prompts)
#   if num_prompts!=0:
#     minibatchsize = int(400/2*num_prompts)

#   if minibatchsize > 56:
#     minibatchsize =56


#   print(f'\n SELECTING top {k} from the beam \n')

#   Nlist=[0.1]*len(list_of_prompts)
#   Qlist=[0]*len(list_of_prompts)


#   promptNQ=[]

#   scored_prompts={}

#   for index in range(len(list_of_prompts)):
#     prompt_dict={}

#     prompt_dict['N']=0.5
#     prompt_dict['Q']=0
#     prompt_dict['prompt']=list_of_prompts[index]
#     prompt_dict['ucb_val']=0

#     promptNQ.append(prompt_dict)

#   #print(promptNQ)
#   for index in range(num_iteration):

#     print(f'********** UCB_E iteration # : {index}','******************')
#     #sample data
#     minibatch_df=sample_n_clauses(df_train,minibatchsize,label)

#     #get top performing prompt: UCB
#     top_prompt=get_argmax_UCB_E(promptNQ)
#     print('TOP PROMPT from UCB_E ',top_prompt)

#     #initilize reward with the f1 score
#     reward=get_score_parallel(top_prompt,label,minibatch_df)
#     top_prompt_index=list_of_prompts.index(top_prompt)

#     print('TOP prompt reward',reward)

#     promptNQ[top_prompt_index]['N']=promptNQ[top_prompt_index]['N']+1
#     promptNQ[top_prompt_index]['Q']=promptNQ[top_prompt_index]['Q']+reward

#   #sort descending and select k best
#   sorted_promptNQ=sorted(promptNQ, key=lambda x:x['Q'],reverse=True)

#   top_k_prompts=[]

#   for index in range(k):

#     top_k_prompts.append(sorted_promptNQ[index]['prompt'])


#   return top_k_prompts

#test
# result=select(["Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?","Is this clause fair in that it states that any judicial proceeding must take place in a location other than the customer\'s residence?"],1)
# print(result)


def select_UCB(list_of_prompts,label,k,minibatchsize,num_iteration):
  num_iteration=2*len(list_of_prompts)
  num_prompts = len(list_of_prompts)
  if num_prompts!=0:
    minibatchsize = int(400/2*num_prompts)

  if minibatchsize > 56:
    minibatchsize =56

  print(f'\n SELECTING top {k} from the beam \n')

  Nlist=[0.1]*len(list_of_prompts)
  Qlist=[0]*len(list_of_prompts)


  promptNQ=[]

  scored_prompts={}

  for index in range(len(list_of_prompts)):
    prompt_dict={}

    prompt_dict['N']=0.5
    prompt_dict['Q']=0
    prompt_dict['prompt']=list_of_prompts[index]
    prompt_dict['ucb_val']=0
    prompt_dict['f1']=0

    promptNQ.append(prompt_dict)

  #print(promptNQ)
  for index in range(num_iteration):

    print(f'********** UCB iteration # : {index}','******************')
    #sample data
    minibatch_df=sample_n_clauses(df_train,minibatchsize,label)

    #get top performing prompt: UCB
    top_prompt=get_argmax_UCB(promptNQ,index)
    print('TOP PROMPT from UCB ',top_prompt)

    #initilize reward with the f1 score
    reward=get_score_parallel(top_prompt,label,minibatch_df)
    top_prompt_index=list_of_prompts.index(top_prompt)

    print('TOP prompt reward',reward)

    promptNQ[top_prompt_index]['N']=promptNQ[top_prompt_index]['N']+1
    promptNQ[top_prompt_index]['Q']=promptNQ[top_prompt_index]['Q']+reward
    promptNQ[top_prompt_index]['f1']=reward

  #sort descending and select k best
  sorted_promptNQ=sorted(promptNQ, key=lambda x:x['Q'],reverse=True)

  top_k_prompts=[]

  # update in top_k_prompts selection
  if len(sorted_promptNQ) > k:

    for index in range(k):
      top_k_prompts.append(sorted_promptNQ[index]['prompt'])
  else:
      top_k_prompts = [item['prompt'] for item in sorted_promptNQ]

  new_list = [{'prompt': d['prompt'], 'f1': d['f1']} for d in sorted_promptNQ] #Remove unnecessary values
  return [top_k_prompts,new_list[0:k]]

#test
# result=select_UCB(["Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?","Is this clause fair in that it states that any judicial proceeding must take place in a location other than the customer\'s residence?","Is there a location mentionned in the clause ?"],'J',2,10,3)
# print(result[1])

In [ ]:
import json

def save_to_file(dict_res,label,method):

  # Convert dictionary to JSON string
  json_str = json.dumps(dict_res, indent=2)

  # Specify the file path
  file_path = f"result_{label}_{method}.json"

  # Write JSON string to a text file
  with open(file_path, 'w') as file:
      file.write(json_str)

  print(f"Dictionary has been saved to {file_path}")

# Main function

In [ ]:
#main function : ALGORITHM 1

def optimization(OGprompt,label,beam_width,search_depth):
  result_dict={}
  #initial prompt in beam
  beam=[OGprompt]
  print("Beam Initialization : ", beam)

  #iterate till search depth
  for i in range(0,search_depth):
    dummy=[]
    print(f'\n ################### Iteration {i+1} of {search_depth} ################\n')

    #iterrate through all prompts in beam so far
    for prompt in beam:

      #fetch candidate prompts
      results=expand_parallel(prompt,20,label,df_train)
      #print('############################',result,'############################')


      #add candidates to the beam
      if results:
        for index in range(len(results)):
          dummy.append(results[index])
          #print(f'DUMMY @ {index}',dummy[index])

    for index in range(len(dummy)):
        #print('FLAGGGG')
        beam.append(dummy[index])

    #select top k in the beam and update beam
    UCB=select_UCB(beam,label,beam_width, 20,5)
    beam=UCB[0]
    scored_dict=UCB[1]
    result_dict[f"iteration_{i}"]=scored_dict
    print("New Beam : ",beam)

    print(f"################### END OF ITERATION {i+1} #######################")
  best=select_UCB(beam,label,1, 20,5)
  result_dict["best_prompt"]=best[1]
  save_to_file(result_dict,label,"UCB")
  return best

#test
# result=optimization("Does this clause specify conditions under which the service provider could amend and modify the terms of service and/or the service itself?","CH",1,2)
# print(result)

In [ ]:
result_LTD=optimization('Does this clause stipulate that duties to pay damages by the provider are limited or excluded?',"LTD",3,5)
print(f"RESULT LIMITATION OF LIABILITY : {result_LTD}")
result_TER=optimization('Does this clause stipulate that the service provider may suspend or terminate the service at any time for any or no reasons and/or without notice?','TER',3,5)
print(f"RESULT UNILATERAL TERMINATION : {result_TER}")
result_USE=optimization('Does this clause stipulate that the consumer is bound by the terms of use of a specific service, simply by using the service, without even being required to mark that he or she has read and accepted them?','USE',3,5)
print(f"RESULT AGREEMENT BY USING : {result_USE}")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

UCB value for prompt:  1. Is it necessary for a user to explicitly acknowledge and accept a clause before they can use a service?
2. If so, does such a clause have the power to obligate the user to the specific terms of the service, ir is 2.9646076147350224

UCB value for prompt:  Here's a new instruction:

Is it necessary to explicitly consent to a service's terms and conditions in order for them to become binding? Explain.

What if we clarify that indicating consent to a service's terms is 2.9646076147350224

UCB value for prompt:  "Does indicating agreement to a service's terms and conditions upon use of the service constitute consent? If agreed upon by a user, is such an action considered to denote acceptance of changes in the company's terms and conditions?" (Correct is 2.9646076147350224

UCB value for prompt:  How does your language model interpret the clause indicating agreement through use of a serv

In [ ]:
print(f"RESULT LIMITATION OF LIABILITY : {result_LTD}")
print(f"RESULT UNILATERAL TERMINATION : {result_TER}")
print(f"RESULT AGREEMENT BY USING : {result_USE}")

RESULT LIMITATION OF LIABILITY : [[' "Analyze the terms of service clause to determine whether it imposes limitations or exclusions on financial liabilities for the service provider. Provide specific examples of potentially unfair clauses and segment the analysis into different parts for better organization. Does this clause'], [{'prompt': ' "Analyze the terms of service clause to determine whether it imposes limitations or exclusions on financial liabilities for the service provider. Provide specific examples of potentially unfair clauses and segment the analysis into different parts for better organization. Does this clause', 'f1': 0.8253968253968254}]]
RESULT UNILATERAL TERMINATION : [[' Revised Instruction:\n\nIs there a provision in the online terms of service clause that allows termination of service without prior notice? What are the conditions under which such termination is allowed?'], [{'prompt': ' Revised Instruction:\n\nIs there a provision in the online terms of service cl